# ***Creating Noisy images with sigma = 15***

In [5]:
import os
import glob
import numpy as np
from PIL import Image
from google.colab import drive

In [6]:
# Define dataset paths
base_path = '/content/drive/MyDrive/optimaldataset'
splits = {
    'Train': os.path.join(base_path, 'Train'),
    'Validate': os.path.join(base_path, 'Validate'),
    'Test': os.path.join(base_path, 'Test')
}
sigma = 15
image_size = (256, 256)

In [8]:
# Verify folder existence
for split, path in splits.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"Folder not found: {path}")
    print(f"Found {split} folder: {path}")


Found Train folder: /content/drive/MyDrive/optimaldataset/Train
Found Validate folder: /content/drive/MyDrive/optimaldataset/Validate
Found Test folder: /content/drive/MyDrive/optimaldataset/Test


In [9]:
# Function to add Gaussian noise
def add_gaussian_noise(image, sigma):
    img_array = np.array(image).astype(np.float32) / 255.0
    noise = np.random.normal(0, sigma / 255.0, img_array.shape)
    noisy_img = np.clip(img_array + noise, 0, 1) * 255.0
    return Image.fromarray(noisy_img.astype(np.uint8))

In [ ]:
# Process each split
for split in splits:
    clean_path = splits[split]
    clean_files = sorted(glob.glob(os.path.join(clean_path, '**', '*.jpg'), recursive=True))
    if not clean_files:
        raise ValueError(f"No .jpg images found in {clean_path}")
    print(f"Processing {split} with sigma={sigma} ({len(clean_files)} images)...")

    for clean_file in clean_files:
        clean_img = Image.open(clean_file).convert('RGB').resize(image_size, Image.Resampling.LANCZOS)
        noisy_img = add_gaussian_noise(clean_img, sigma)
        rel_path = os.path.relpath(clean_file, clean_path)
        noisy_dir = os.path.join(clean_path, f'noisy_sigma{sigma}', os.path.dirname(rel_path))
        os.makedirs(noisy_dir, exist_ok=True)
        output_file = os.path.join(noisy_dir, os.path.basename(clean_file).rsplit('.', 1)[0] + '.png')
        noisy_img.save(output_file, 'PNG')
        clean_output_file = os.path.join(clean_path, rel_path.rsplit('.', 1)[0] + '.png')
        os.makedirs(os.path.dirname(clean_output_file), exist_ok=True)
        clean_img.save(clean_output_file, 'PNG')

    print(f"Saved {len(clean_files)} noisy (sigma={sigma}) and clean .png images to {clean_path}")

print("All done! Noisy and clean RGB images (256x256, sigma=15) saved as .png in each split.")